In [ ]:
#pip install -r requirements.txt

In [13]:
import warnings
warnings.filterwarnings('ignore')

#importing libraries
import pandas as pd
import numpy as np
import re
from collections import defaultdict

#importing spacy libraries
import spacy
nlp = spacy.load("en_core_web_sm")

#import nltk libraries
import nltk
nltk.download
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize, pos_tag, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [15]:
tweetData = pd.read_json("/content/drive/MyDrive/Colab Notebooks/Jio Institute/NLP/ggg2013.json")
tweetData.head()

,text,user,id,timestamp_ms
0,JLo's dress! #eredcarpet #GoldenGlobes,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38
1,What's making Sofia Vergara's boobs stay like ...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38
3,Anne Hathaway has got me living. #GoldenGlobes,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38
4,Jennifer Lopez's lace dress? Thoughts? #Golden...,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38


In [16]:
#finds all hashtags and add to hashtag column
tweetData["hashtag"] = tweetData["text"].apply(lambda x: re.findall(r"#(\w+)",x))

#finds all mentions and add to mention column
tweetData["mention"] = tweetData["text"].apply(lambda x: re.findall(r"@(\w+)",x))

#Drop duplicate columns
tweetData.drop_duplicates(subset="text",inplace=True)

#remove urls
tweetData['text'] = tweetData['text'].str.replace('https?://\S+|www\.\S+|ftp://\S+', '')
tweetData['text'] = tweetData['text'].str.replace('GoldenGlobes', '')
tweetData['text'] = tweetData['text'].str.replace('[G|g]olden\\s?[G|g]lobes*', '')

#remove numbers
tweetData['text'] = tweetData['text'].astype(str).str.replace('\d+', '')

#remove punctuation
tweetData['text'] = tweetData['text'].str.replace('[^\w\s]','')

#remove unneccesary hashtags
tweetData['text'] = tweetData['text'].str.replace('#|@|RT', '')

#Removing duplicate Tweets
tweetData.drop_duplicates(subset=['text'], keep='first')

tweetData.head()

,text,user,id,timestamp_ms,hashtag,mention
0,JLo's dress! #eredcarpet #,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38,"[eredcarpet, GoldenGlobes]",[]
1,What's making Sofia Vergara's boobs stay like ...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38,[GoldenGlobes],[]
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38,[goldenglobes],"[FabSugar, goldenglobes]"
3,Anne Hathaway has got me living. #,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38,[GoldenGlobes],[]
4,Jennifer Lopez's lace dress? Thoughts? #,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38,[GoldenGlobes],[]


In [17]:
#lemmatize text
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer =  TweetTokenizer()

def lemmatize_text(text):
  return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]

#remove any punctuations
def remove_punctuation(words):
  new_words = []
  for word in words:
    new_word = re.sub(r'[^\w\s]','', (word))
    if new_word != '':
        new_words.append(new_word)
  return new_words

words = tweetData['text'].apply(lemmatize_text)
tweetData["bagOfWords"] = words.apply(remove_punctuation)
tweetData.head()

,text,user,id,timestamp_ms,hashtag,mention,bagOfWords
0,JLo's dress! #eredcarpet #,"{'screen_name': 'Dozaaa_xo', 'id': 557374298}",290620657987887104,2013-01-14 00:45:38,"[eredcarpet, GoldenGlobes]",[],"[JLos, dress, eredcarpet]"
1,What's making Sofia Vergara's boobs stay like ...,"{'screen_name': 'theAmberShow', 'id': 14648726}",290620657887219713,2013-01-14 00:45:38,[GoldenGlobes],[],"[Whats, making, Sofia, Vergaras, boob, stay, l..."
2,RT @FabSugar: Kerry Washington is EVERYTHING. ...,"{'screen_name': 'SweetyPW', 'id': 35498686}",290620657828524032,2013-01-14 00:45:38,[goldenglobes],"[FabSugar, goldenglobes]","[RT, FabSugar, Kerry, Washington, is, EVERYTHI..."
3,Anne Hathaway has got me living. #,"{'screen_name': '_NicoleEdwards', 'id': 144430...",290620657799159809,2013-01-14 00:45:38,[GoldenGlobes],[],"[Anne, Hathaway, ha, got, me, living]"
4,Jennifer Lopez's lace dress? Thoughts? #,"{'screen_name': 'lolaogunnaike', 'id': 134953223}",290620657778188288,2013-01-14 00:45:38,[GoldenGlobes],[],"[Jennifer, Lopezs, lace, dress, Thoughts]"


In [18]:
df_awards = pd.DataFrame()
df_awards['newText'] = tweetData["bagOfWords"].apply(lambda x:' '.join(x))
df_awards.head(2)

,newText
0,JLos dress eredcarpet
1,Whats making Sofia Vergaras boob stay like tha...


In [ ]:
'''df_awards = df_awards[df_awards['newText'].str.contains("Best ")].reset_index(drop=True)
df_awards['nouns'] = df_awards['newText'].apply(lambda x: [*nlp(x).ents])

#removing rows with empty list of nouns
df_awards = df_awards[df_awards['nouns'].apply(lambda x: len(x)>1)]

#Creating New dataframe to store all nouns as separate to get count
awardNames =[str(j) for i in df_awards['nouns'] for j in i]
dfAwardNames = pd.DataFrame(awardNames,columns=['values'])'''

'df_awards = df_awards[df_awards[\'newText\'].str.contains("Best ")].reset_index(drop=True)\ndf_awards[\'nouns\'] = df_awards[\'newText\'].apply(lambda x: [*nlp(x).ents])\n\n#removing rows with empty list of nouns\ndf_awards = df_awards[df_awards[\'nouns\'].apply(lambda x: len(x)>1)]\n\n#Creating New dataframe to store all nouns as separate to get count\nawardNames =[str(j) for i in df_awards[\'nouns\'] for j in i]\ndfAwardNames = pd.DataFrame(awardNames,columns=[\'values\'])'

In [20]:
newDF = pd.DataFrame()
newDF['text'] = tweetData['text'].str.lower()

def extract_award_name(text):
    pattern = re.compile(r'best (?:actress|actor|screenplay|motion picture|director|screenplay|original score|original song|animated feature film)(?: (?:in a )?(?:motion picture|mini-series|tv|movie|film|drama|comedy or musical))?', re.IGNORECASE)
    matches = pattern.findall(text)
    return ', '.join(set(matches)) if matches else None

df = pd.DataFrame(newDF['text'])
# Apply function to DataFrame
df['award_names'] = newDF['text'].apply(extract_award_name)

# Filter rows with extracted award names
df_with_awards = df[df['award_names'].notna()]
df_with_awards['award_names'].value_counts()

award_names
best actor                                                     559
best director                                                  558
best actress                                                   490
best original song                                             359
best motion picture                                            354
best screenplay                                                262
best actor in a motion picture                                 161
best original score                                            142
best actress in a motion picture                               129
best actress in a tv                                           122
best animated feature film                                     110
best actor in a tv                                             108
best motion picture drama                                       86
best actor tv                                                   27
best actress in a comedy or musical               

In [21]:
top10awards = df_with_awards['award_names'].value_counts()[:10].index.tolist()

In [31]:
def returnWordCnts(findStr,df_awards):
  df_awards = df_awards[df_awards['newText'].str.contains(findStr)].reset_index(drop=True)
  df_awards['nouns'] = df_awards['newText'].apply(lambda x: [*nlp(x).ents])

  #removing rows with empty list of nouns
  df_awards = df_awards[df_awards['nouns'].apply(lambda x: len(x)>1)]

  #Creating New dataframe to store all nouns as separate to get count
  NewList =[str(j) for i in df_awards['nouns'] for j in i]
  dfBestActreesNewDf = pd.DataFrame(NewList,columns=['values'])

  return dfBestActreesNewDf.value_counts()

#**Results**

In [ ]:
#Names of the award are
for i in top10awards:
  print(i)

best actor
best director
best actress
best original song
best motion picture
best screenplay
best actor in a motion picture
best original score
best actress in a motion picture
best actress in a tv


In [44]:
#Getting Host Names
def getHosts():
  hosts = returnWordCnts("host|Host",df_awards)
  hostsL = hosts[:2].index.tolist()
  hostsL = [i[0] for i in hostsL]
  hostss = ' and '.join(hostsL)
  return hostss

Hostsss = getHosts()

In [33]:
#Getting at most 5 nominees for each awards
def getAwardNominees(top10):
  newAwardL = []
  for eachAward in top10:
    awrd = returnWordCnts(eachAward,df_awards)
    awrdL = awrd[:5].index.tolist()
    awrdsL = [i[0] for i in awrdL]
    awrdss = ' , '.join(awrdsL)
    newAwardL.append(awrdsL)
  return newAwardL
  print("Nominees of the award",eachAward ," are :", awrdss)
listOfNominees = getAwardNominees(top10awards)

In [34]:
listOfNominees

[['Hugh Jackman', 'Lincoln', 'Daniel DayLewis', 'Les Miserables', 'Les Mis'],
 ['Ben Affleck',
  'Argo USA TODAY',
  'Golden Globes Affleck',
  'San Francisco',
  'Oscar'],
 ['Jennifer Lawrence',
  'Jessica Chastain',
  'Silver Linings Playbook',
  'Julianne Moore',
  'Claire Danes'],
 ['Adele', 'Skyfall', 'OfficialAdele', 'Taylor Swift', 'Daniel Craig'],
 ['the Golden Globes',
  'Quentin Tarantino',
  'Les Miserables',
  'Lincoln',
  'Django'],
 ['Quentin Tarantino',
  'Django',
  'Django Unchained',
  'DjangoUnchained',
  'Tarantino'],
 ['Daniel DayLewis',
  'Les Miserables',
  'Hugh Jackman',
  'Lincoln',
  'LincolnMovie httptcoBshI8XDL'],
 ['GreggPearson',
  'LIFE OF PI',
  'Life Of Pi Michael Danna',
  'Life of Pi Mychael Danna',
  'Michael Danna'],
 ['Jessica Chastain',
  'the Golden Globes',
  'KNXDavid Jennifer Lawrence',
  'NewsGunner',
  'Silver Linings Playbook'],
 ['Claire Danes', 'Congrats']]

In [41]:
#Get winners for each awards
def getAwardandWinners(top10):
  newAwardL = []
  for eachAward in top10:
    awrd = returnWordCnts(eachAward,df_awards)
    awrdL = awrd[:2].index.tolist()
    newAwardL.append(awrdL[0][0])
  return newAwardL
awardWinnerList = getAwardandWinners(top10awards)
awardWinnerList

['Hugh Jackman',
 'Ben Affleck',
 'Jennifer Lawrence',
 'Adele',
 'the Golden Globes',
 'Quentin Tarantino',
 'Daniel DayLewis',
 'GreggPearson',
 'Jessica Chastain',
 'Claire Danes']

In [67]:
#Getting List of Award Presenters
def getAwardPresenter():
  hosts = returnWordCnts("award presenters|Award presenters|award presenter|Award presenter",df_awards)
  hostsL = hosts[2:].index.tolist()
  hostsL = [i[0] for i in hostsL]
  hostss = ' and '.join(hostsL)
  return hostss

In [54]:
#Getting Host Names
def getBestDressed():
  hosts = returnWordCnts("best dressed tonight|Best Dressed Tonight|Best dressed tonight",df_awards)
  hostsL = hosts[:10].index.tolist()
  hostsL = [i[0] for i in hostsL]
  return hostsL[1]

In [58]:
def getWorstDressed():
  hosts = returnWordCnts("worst dressed|Worst Dressed|Worst dressed",df_awards)
  hostsL = hosts[:10].index.tolist()
  hostsL = [i[0] for i in hostsL]
  return hostsL[3]

getWorstDressed()

'Sienna Miller'

In [60]:
def mostControversial():
  hosts = returnWordCnts("Controversial|controversial",df_awards)
  hostsL = hosts[:10].index.tolist()
  hostsL = [i[0] for i in hostsL]
  return hostsL[2]

mostControversial()

'Ben Afflecks'

In [69]:
print("Hosts of this event are :",Hostsss,'\n')
print("Presenters of the awards are ",getAwardPresenter(),'\n')
for i,j in enumerate(top10awards):
  print("Nominees of the award",j ," are :", ' '.join(listOfNominees[i]))
  print("Winner of the award",j ," is :", awardWinnerList[i],"\n")

print("Best dressed :",getBestDressed(),'\n')
print("Worst dressed :",getWorstDressed(),'\n')
print("Most controversial :",mostControversial(),'\n')

Hosts of this event are : Amy Poehler and Tina Fey 

Presenters of the awards are  Michael J Foxs and RT Just_Adly 

Nominees of the award best actor  are : Hugh Jackman Lincoln Daniel DayLewis Les Miserables Les Mis
Winner of the award best actor  is : Hugh Jackman 

Nominees of the award best director  are : Ben Affleck Argo USA TODAY Golden Globes Affleck San Francisco Oscar
Winner of the award best director  is : Ben Affleck 

Nominees of the award best actress  are : Jennifer Lawrence Jessica Chastain Silver Linings Playbook Julianne Moore Claire Danes
Winner of the award best actress  is : Jennifer Lawrence 

Nominees of the award best original song  are : Adele Skyfall OfficialAdele Taylor Swift Daniel Craig
Winner of the award best original song  is : Adele 

Nominees of the award best motion picture  are : the Golden Globes Quentin Tarantino Les Miserables Lincoln Django
Winner of the award best motion picture  is : the Golden Globes 

Nominees of the award best screenplay  ar